In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True '
os.environ['TORCH_USE_CUDA_DSA'] = 'True '
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [1]:
 # "excellent-shard-422915-n5"
from google.colab import auth

# PROJECT_ID = "excellent-shard-422915-n5"  # @param {type:"string"}

auth.authenticate_user()

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -

!apt -qq update

!apt -qq install gcsfuse

!mkdir colab_directory

!gcsfuse --implicit-dirs testopolito colab_directory

!ls colab_directory

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  10466      0 --:--:-- --:--:-- --:--:-- 10509
OK
57 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 57 not upgraded.
Need to get 10.4 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.0.1_amd64.deb ...
Unpacking gcsfuse (2.0.1) ...
Setting up gcsfuse (2.0.1) ...
{"timestamp":{"seconds":1716457560,"nanos"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
import numpy as np
import random
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [3]:
# Definizione del generatore U-Net
class UNetGenerator(nn.Module):
    def __init__(self, input_channels=6, output_channels=3):
        super(UNetGenerator, self).__init__()

        # Encoder
        self.enc1 = self.block(input_channels, 64, kernel_size=4, stride=2, padding=1, batch_norm=False)
        self.enc2 = self.block(64, 128, kernel_size=4, stride=2, padding=1)
        self.enc3 = self.block(128, 256, kernel_size=4, stride=2, padding=1)
        self.enc4 = self.block(256, 512, kernel_size=4, stride=2, padding=1)
        self.enc5 = self.block(512, 512, kernel_size=4, stride=2, padding=1)
        self.enc6 = self.block(512, 512, kernel_size=4, stride=2, padding=1)
        self.enc7 = self.block(512, 512, kernel_size=4, stride=2, padding=1)
        self.enc8 = self.block(512, 512, kernel_size=4, stride=2, padding=1, batch_norm=False)

        # Decoder
        self.dec1 = self.block(512, 512, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec2 = self.block(1024, 512, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec3 = self.block(1024, 512, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec4 = self.block(1024, 512, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec5 = self.block(1024, 256, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec6 = self.block(512, 128, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec7 = self.block(256, 64, kernel_size=4, stride=2, padding=1, batch_norm=True, transpose=True)
        self.dec8 = self.block(128, output_channels, kernel_size=4, stride=2, padding=1, batch_norm=False, transpose=True)

        self.tanh = nn.Tanh()

    def block(self, in_channels, out_channels, kernel_size, stride, padding, batch_norm=True, transpose=False):
        layers = []
        if transpose:
            layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False))
        else:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False))
        if batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)
        enc5 = self.enc5(enc4)
        enc6 = self.enc6(enc5)
        enc7 = self.enc7(enc6)
        enc8 = self.enc8(enc7)

        # Decoder
        dec1 = self.dec1(enc8)
        dec1 = torch.cat([dec1, enc7], dim=1)
        dec2 = self.dec2(dec1)
        dec2 = torch.cat([dec2, enc6], dim=1)
        dec3 = self.dec3(dec2)
        dec3 = torch.cat([dec3, enc5], dim=1)
        dec4 = self.dec4(dec3)
        dec4 = torch.cat([dec4, enc4], dim=1)
        dec5 = self.dec5(dec4)
        dec5 = torch.cat([dec5, enc3], dim=1)
        dec6 = self.dec6(dec5)
        dec6 = torch.cat([dec6, enc2], dim=1)
        dec7 = self.dec7(dec6)
        dec7 = torch.cat([dec7, enc1], dim=1)
        dec8 = self.dec8(dec7)

        return self.tanh(dec8), enc8

In [4]:
# Function to combine generated parts with the original image
def combine_images(original, generated, mask):
    return generated * mask + original * (1 - mask)

In [5]:
# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

In [6]:
# Utility function to create a masked image with random position and size
def create_random_masked_image(image):
    _, _, height, width = image.size()
    mask = torch.zeros_like(image)

    # Define random position and size
    top = random.randint(0, height // 2)
    left = random.randint(0, width // 2)
    patch_height = random.randint(height // 4, height // 2)
    patch_width = random.randint(width // 4, width // 2)

    # Apply mask
    mask[:, :, top:top + patch_height, left:left + patch_width] = 1
    masked_image = image.clone()
    masked_image[:, :, top:top + patch_height, left:left + patch_width] = 0

    return masked_image, mask

In [7]:
# Load and preprocess the image
def load_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

In [8]:
# Function to convert a tensor to a PIL image and display it
def tensor_to_pil_image(tensor):
    tensor = tensor.clone().detach().cpu()
    tensor = tensor.squeeze(0)  # remove batch dimension
    tensor = transforms.Normalize((-1, -1, -1), (2, 2, 2))(tensor)  # unnormalize
    tensor = tensor.permute(1, 2, 0)  # convert to HWC format
    image = tensor.numpy()
    image = np.clip(image, 0, 1)
    return Image.fromarray((image * 255).astype(np.uint8))

In [9]:
# Initialize the generator and discriminator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = UNetGenerator().to(device)
discriminator = Discriminator().to(device)

In [ ]:
# Load and mask the image
image_path = "/content/colab_directory/CRC_WSIs_original/in_roi_patches/1.svs/10240_10240_mag1.png"
image = load_image(image_path).to(device)
masked_image, mask = create_random_masked_image(image)

# Concatenate masked image and mask along the channel dimension
z = torch.cat((masked_image, mask), dim=1)

In [83]:
# Display masked image
masked_pil_image = tensor_to_pil_image(masked_image)
masked_pil_image.show()  # This will open the image in the default image viewer
masked_pil_image.save("masked_image.png")  # Optionally save the image to disk

In [10]:
# GAN training setup
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.00005, betas=(0.5, 0.999))
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.00005, betas=(0.5, 0.999))

scheduler_g = torch.optim.lr_scheduler.StepLR(optimizer_g, step_size=30, gamma=0.1)
scheduler_d = torch.optim.lr_scheduler.StepLR(optimizer_d, step_size=30, gamma=0.1)

# Temporary dataset with patient 1 only

In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import re
import numpy as np

class patchesDataset(Dataset):
    def __init__(self, root_dir, patient_id, transform=None):
        self.root_dir = root_dir
        self.patient_id = patient_id
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.patients = []
        self.coordinates = []

        # Process not_roi_patches (label 0)
        not_roi_dir = os.path.join(root_dir, 'not_roi_patches')
        patient_dir = os.path.join(not_roi_dir, f'{patient_id}.svs')
        for img_name in os.listdir(patient_dir):
            self.image_paths.append(os.path.join(patient_dir, img_name))
            self.labels.append(0)
            self.patients.append(patient_id)
            self.coordinates.append(self._extract_coordinates(img_name))

        # Process in_roi_patches (label 1)
        in_roi_dir = os.path.join(root_dir, 'in_roi_patches')
        patient_dir = os.path.join(in_roi_dir, f'{patient_id}.svs')
        for img_name in os.listdir(patient_dir):
            self.image_paths.append(os.path.join(patient_dir, img_name))
            self.labels.append(1)
            self.patients.append(patient_id)
            self.coordinates.append(self._extract_coordinates(img_name))

    def _extract_coordinates(self, img_name):
        # Extract x and y from the filename
        match = re.match(r'(\d+)_(\d+)_.*\.png', img_name)
        if match:
            x, y = int(match.group(1)), int(match.group(2))
            return (x, y)
        else:
            raise ValueError(f"Filename {img_name} does not match the expected pattern.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGBA")  # Ensure image is RGBA
        image = np.array(image)[:, :, :3]  # Drop the alpha channel
        image = Image.fromarray(image)  # Convert back to PIL image
        label = self.labels[idx]
        patient_id = self.patients[idx]
        coordinates = self.coordinates[idx]

        if self.transform:
            image = self.transform(image)
        return image, label, patient_id, coordinates

In [12]:
# Specify the patient ID you want to process
patient_id = 1

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset_temp = patchesDataset(root_dir="E:\PoliTo\Corsi PoliTo\Semestre 4\Machine learning in applications\Progetto\Dataset patches\content\colab_directory\CRC_WSIs_original\\", patient_id=patient_id, transform=transform)
dataloader_temp = DataLoader(dataset_temp, batch_size=8, shuffle=True)

# Example of iterating through the dataloader
for images, labels, patient_ids, coords in dataloader_temp:
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels: {labels}')
    print(f'Batch of patient IDs: {patient_ids}')
    print(f'Batch of coordinates: {coords}')
    break  # Remove this break to iterate through all batches

Batch of images shape: torch.Size([8, 3, 512, 512])
Batch of labels: tensor([1, 0, 1, 1, 1, 0, 0, 1])
Batch of patient IDs: tensor([1, 1, 1, 1, 1, 1, 1, 1])
Batch of coordinates: [tensor([14336, 17408,  8192, 16896,  5120,  6656, 21504, 20480]), tensor([13824,  5632, 16896, 16384, 16384, 11776,  3072, 12288])]


In [13]:
# Inizializza il dataloader per il paziente 1
patient_id = 1
dataset = patchesDataset(root_dir='E:\PoliTo\Corsi PoliTo\Semestre 4\Machine learning in applications\Progetto\Dataset patches\content\colab_directory\CRC_WSIs_original\\', patient_id=patient_id, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [14]:
def print_memory_usage():
    print(f'Memory Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB')
    print(f'Memory Cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB')

num_epochs = 5

embeddings_list = []
labels_list = []
batch_size = 8
# epoch = 0
# Training loop
for epoch in range(num_epochs):
    for images, labels, patient_ids, coords in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Create random masked image
        masked_images = []
        masks = []
        for image in images:
            masked_image, mask = create_random_masked_image(image.unsqueeze(0))
            masked_images.append(masked_image)
            masks.append(mask)

        masked_images = torch.cat(masked_images).to(device)
        masks = torch.cat(masks).to(device)

        # Train Discriminator
        optimizer_d.zero_grad()
        real_labels = torch.full((batch_size,), 0.9, device=device)
        fake_labels = torch.full((batch_size,), 0.1, device=device)


        outputs = discriminator(images).view(images.size(0), -1).mean(1)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.cat((images, masks), dim=1)  # Concatenate the whole image with the mask
        generated_parts, embedding = generator(z)
        fake_images = combine_images(images, generated_parts, masks)
        outputs = discriminator(fake_images.detach()).view(images.size(0), -1).mean(1)
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        optimizer_d.step()

        # Train Generator
        optimizer_g.zero_grad()
        outputs = discriminator(fake_images).view(images.size(0), -1).mean(1)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()

        optimizer_g.step()

        # Collect embeddings and labels for SVM training
        embeddings_list.append(embedding.view(embedding.size(0), -1).cpu().detach().numpy())
        labels_list.append(real_labels.cpu().detach().numpy())
        embeddings_list.append(embedding.view(embedding.size(0), -1).cpu().detach().numpy())
        labels_list.append(fake_labels.cpu().detach().numpy())

        print(f'Epoch [{epoch}/{num_epochs}], d_loss: {d_loss_real + d_loss_fake:.4f}, g_loss: {g_loss:.4f}')
        save_image(fake_images, f'output_{epoch}.png')

        # Debug: print memory usage
        print_memory_usage()

        # Clear cache
        del masked_images, masks, outputs, fake_images, z, embedding, generated_parts
        torch.cuda.empty_cache()
        scheduler_g.step()
        scheduler_d.step()

# Save embeddings and labels periodically
embeddings_list = np.concatenate(embeddings_list)
labels_list = np.concatenate(labels_list)
np.save('embeddings_list.npy', embeddings_list)
np.save('labels_list.npy', labels_list)


Epoch [0/5], d_loss: 1.4042, g_loss: 0.6999
Memory Allocated: 1048.68 MB
Memory Cached: 2904.00 MB
Epoch [0/5], d_loss: 1.4655, g_loss: 0.6728
Memory Allocated: 1047.68 MB
Memory Cached: 3556.00 MB
Epoch [0/5], d_loss: 1.4108, g_loss: 0.6802
Memory Allocated: 1047.68 MB
Memory Cached: 3540.00 MB
Epoch [0/5], d_loss: 1.3967, g_loss: 0.6813
Memory Allocated: 1047.68 MB
Memory Cached: 3532.00 MB
Epoch [0/5], d_loss: 1.3913, g_loss: 0.6863
Memory Allocated: 1048.68 MB
Memory Cached: 3552.00 MB
Epoch [0/5], d_loss: 1.3860, g_loss: 0.6903
Memory Allocated: 1048.68 MB
Memory Cached: 3552.00 MB
Epoch [0/5], d_loss: 1.3849, g_loss: 0.6872
Memory Allocated: 1048.68 MB
Memory Cached: 3552.00 MB
Epoch [0/5], d_loss: 1.3831, g_loss: 0.6966
Memory Allocated: 1048.68 MB
Memory Cached: 3552.00 MB
Epoch [0/5], d_loss: 1.3870, g_loss: 0.6990
Memory Allocated: 1048.68 MB
Memory Cached: 3552.00 MB
Epoch [0/5], d_loss: 1.3900, g_loss: 0.6873
Memory Allocated: 1048.68 MB
Memory Cached: 3552.00 MB
Epoch [0/5

ValueError: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([5])) is deprecated. Please ensure they have the same size.

In [23]:
# Save the GAN models
torch.save({
    'generator_state_dict': generator.state_dict(),
    'discriminator_state_dict': discriminator.state_dict(),
    'optimizerG_state_dict': optimizer_g.state_dict(),
    'optimizerD_state_dict': optimizer_d.state_dict(),
}, 'gan_random_masks.pth')

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [ ]:
print(embeddings_list.shape)
print(labels_list.shape)

In [ ]:
# Save the final generated image
save_image(fake_images, 'final_output.png')

In [15]:
from sklearn.decomposition import PCA

# Carica embeddings e labels
embeddings_list = np.load('embeddings_list.npy')
labels_list = np.load('labels_list.npy')
"""
# Riduci la dimensionalità con PCA
n_components = 100  # Puoi regolare questo numero in base alla tua esigenza
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(embeddings_list)
"""
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(embeddings_list, labels_list, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))